In [1]:
# Dependências
from sequence.protocol import Protocol
from sequence.message import Message
from enum import Enum, auto
from sequence.kernel.timeline import Timeline
from sequence.components.memory import MemoryArray
from sequence.topology.node import Node, QuantumRouter
from sequence.resource_management.resource_manager import ResourceManager

In [2]:
# Estados possíveis do protocolo de entrelaçamento com handshake
class HandshakeState(Enum):
    CLOSED = auto()
    SYN_SENT = auto()
    SYN_RECEIVED = auto()
    ESTABLISHED = auto()
    FINISHED = auto()
    FAILED = auto()

# Tipos de mensagem do protocolo de entrelaçamento
class HandshakeMessageType(Enum):
    SYN = auto()
    SYN_ACK = auto()
    ACK = auto()
    
# Mensagem de hanshake do protocolo de entrelaçamento
class EntangleConnectionHandshakeMessage(Message):
    """
    A mensagem é composta por um message_type (SYN, SYN_ACK, ACK), um session_id (id da sessão/tentativa de entrelaçamento) e um memory_name (id da memória que vai ser usada).
    """
    # Protocol_receiver é o protocolo que vai receber a mensagem no remote_node, quando o owner envia a mensagem.
    # Ele identifica por if protocol.name == msg.receiver
    # O memory_name é o id da memória que o owner do protocolo vai usar para o entrelaçamento, independente se é o A ou B
    def __init__(self, msg_type, session_id, protocol_receiver, memory_name=None):
        super().__init__(msg_type, protocol_receiver)
        self.session_id = session_id
        self.memory_name = memory_name

    def __str__(self):
        return f"{self.msg_type.name} | session: {self.session_id} | mem: {self.memory_name}"

In [3]:
class EntangleConectionHandshakeProtocol(Protocol):
    """
    Protocolo de entrelaçamento entre dois nós. A e B serão entrelaçados, sendo A o nó que solicita o entrelaçamento e B o nó que responde. O protocolo funciona com um handshake de três etapas: SYN, SYN_ACK e ACK.
    
    """
    # O session_id serve para identificar a sessão de entrelaçamento, ela será única para cada tentativa em par. Ou seja, o session_id é o mesmo na instância do protocolo de A e B.
    # O name é o nome do protocolo, que é uma string que identifica o protocolo, serve para o roteamento das mensagens. Ele é composto pelo nome do owner, o nome do remote_node e o session_id. Será o mesmo para os dois protocolos, A e B.
    # O owner é o nó que tem o protocolo, o "dono" da instância, mesmo que seja quem recebeu ou enviou a solicitação de entrelaçamento
    # O remote_node é o nó que vai receber a solicitação de entrelaçamento, mas ele também tem uma instância onde ele é o owner do protocolo
    # O target_memory é a memória que vai ser usada para o entrelaçamento, ela é alocada quando o protocolo é iniciado
    def __init__(self, owner, remote_node, session_id=None):
        
        if session_id is None:
            self.session_id = f'{id(self)}'
        else: 
            self.session_id = session_id
            
        name = f"ech[{owner.name}]2[{remote_node}]{self.session_id}"
        
        super().__init__(owner, name)
        self.remote_node = remote_node
        self.target_memory = None
    
    def start(self):
        """
        Inicia o protocolo de entrelaçamento, entre A e B. Sendo A o que solicita o entrelaçamento e B o que responde.
        """
        # Cria a mensagem de SYN
        msg = EntangleConnectionHandshakeMessage(
            msg_type=HandshakeMessageType.SYN,
            session_id=self.session_id,
            protocol_receiver=f"ech2[{self.remote_node.name}]"
        )
        
        # Envia a mensagem de SYN para o nó remoto
        self.owner.send_message(self.remote_node.name, msg)
        
        # Muda o próprio estado para SYN_SENT
        self.set_state(HandshakeState.SYN_SENT)
    
    def send_message(self, dest, msg_type):
        """
        Envia a mensagem para o remote_node.
        
        Args:
            dest (str): O nó de destino.
            msg_type (Enum): O tipo da mensagem.
        """
        # TODO: Basear a lógica do envio das mensagens no estado do protocolo.
        # marca o protocolo que vai receber a mensagem
        receiver_protocol = f"ech[{self.owner.name}]2[{self.remote_node.name}]{self.session_id}"
        
        # cria a mensagem de handshake
        msg = EntangleConnectionHandshakeMessage(
            msg_type=msg_type,
            session_id=self.session_id,
            protocol_receiver=receiver_protocol,
            memory_name=self.target_memory.name if msg_type != HandshakeMessageType.SYN else None
        )
        
        # envia a mensagem para o nó de destino
        self.owner.send_message(dest, msg)
        
        
    def received_message(self, src, msg):
        """
        Recebe a mensagem do remote_node e processa de acordo com o estado do protocolo.
        """
        if msg.msg_type == HandshakeMessageType.SYN:      
            # então sou um B, tem um hub querendo se conectar
            # aloca a memória que vai ser usada para o entrelaçamento
            self.target_memory = self.owner.allocate_raw_memory()
            
            # se nenhuma memória disponível
            if self.target_memory is None:
                print("Nenhuma memória disponível")
                return
            
            # responde com um SYN_ACK
            self.send_message(src, HandshakeMessageType.SYN_ACK)
            
            # muda o estado para SYN_RECEIVED
            self.set_state(HandshakeState.SYN_RECEIVED)
            
        elif msg.msg_type == HandshakeMessageType.SYN_ACK:
            # então sou um A, já enviei o SYN e recebi o SYN_ACK
            self. target_memory = self.owner.allocate_raw_memory(msg.memory_name)
            
            # se não tiver memória disponível, então encerra o protocolo
            if self.target_memory is None:
                print("Nenhuma memória disponível")
                return
            
            # recebi qual a memória que o receiver quer usar, então posso enviar a mensagem de ACK com o id da memória que vou usar
            # responde com um ACK
            self.send_message(src, HandshakeMessageType.ACK)
            
            print(f"Recebido SYN_ACK de {src} com session_id {msg.session_id} e memory_name {self.target_memory.name}")
            
            # muda o estado para ESTABLISHED
            self.set_state(HandshakeState.ESTABLISHED)
            
        elif msg.msg_type == HandshakeMessageType.ACK:
            # então sou um B, já enviei o SYN_ACK e recebi o ACK
            
            # finaliza o handshake
            self.set_state(HandshakeState.FINISHED)
            print(f"Recebido ACK de {src} com session_id {msg.session_id} e memory_name {self.target_memory.name}")
        
        else:
            # se a mensagem não for do tipo SYN, SYN_ACK ou ACK, então encerra o protocolo
            print("Mensagem inválida")
            self.set_state(HandshakeState.FAILED)
    
    
    def set_state(self, state):
        # se o estado mudar para FAILED então encerra
        # self.close()
        self.state = state
    
    def close(self):
        # TODO: Melhorar isso. Lógica de fechamento do protocolo. O que fazer com a memória? Desalocar? Deixar ocupada?
        if self.target_memory is not None:
            self.set_state(HandshakeState.FAILED)   
            
class EntangleConnectionDispatcherProtocol(Protocol):
    """
    Dispatch do protocolo de entrelaçamento. Ele é responsável por receber as mensagens de solicitação de entrelaçamento (SYN) e criar um protocolo de entrelaçamento para cada um deles. E continuar o protocolo a apartir do SYN recebido encaminhando um SYN_ACK para o nó que solicitou o entrelaçamento. Quando um nó A quer se conectar a um nó B, usa o nome "ech2[B]" como nome do protocolo 'receiver'.
    """
    def __init__(self, owner):
        name = f"ech2[{owner.name}]"
        super().__init__(owner, name)

    def received_message(self, src, msg):
        # TODO: Verificar se não é melhor não checar a memória aqui, e sim só no protocolo de entrelaçamento.
        # Verifica se a mensagem é SYN, se não for, então encerra o protocolo. 
        if msg.msg_type == HandshakeMessageType.SYN:
            memory = self.owner.allocate_raw_memory()
            # verifica se nenhuma memória disponível
            if memory is None:
                print("Nenhuma memória disponível")
                return
        else:
            return
        
        print(f"Recebido SYN de {src} com session_id {msg.session_id}")# e memory_name {memory.name}")                    
        # cria o protocolo de entrelaçamento para o nó que está solicitando o entrelaçamento
        proto_name = f"ech[{src}]2[{self.owner.name}]{msg.session_id}"
        entangle_proto = EntangleConectionHandshakeProtocol(
            owner = self.owner,
            remote_node = src,
            session_id = msg.session_id
        )
        
        # adiciona o protocolo de entrelaçamento na lista de protocolos do owner
        self.owner.protocols.append(entangle_proto)
        
        # envia o SYN para o novo protocolo de entrelaçamento
        new_syn_msg = EntangleConnectionHandshakeMessage(
            msg_type=HandshakeMessageType.SYN_ACK,
            session_id=msg.session_id,
            protocol_receiver=msg.receiver, # ou proto_name
            memory_name=memory.name
        )
        self.owner.send_message(src, new_syn_msg)    

In [4]:
# Hub tem que ter memória, gerenciar os entrelaçamentos
class Hub(QuantumRouter):
    """Nó responsável por gerenciar a memória e os entrelaçamentos. Fica entre os sensores e o gateway de saída."""
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        # Memória de qubits, são esses argumentos
        memory_array_name = f"{self.name}.memory_array"
        # outros detalhes envolvem parametros de configuração da memória, mas por enquanto só isso
        self.memory_array = MemoryArray(memory_array_name, timeline, num_memories=10)
        # adiciona as memórias ao hub
        self.add_component(self.memory_array)
        # indica que a memória é desse hub
        self.memory_array.add_receiver(self)
        # gerenciar os entrelaçamentos (pode ser melhor definir o próprio, mas por enquanto é assim)
        self.resource_manager = ResourceManager(self, memory_array_name)

    def get(self, photon, **kwargs):
        src = kwargs.get("src")
        dst = kwargs.get("dst")

        # se não for o destino, apenas reencaminha o photon para o destino correto
        if dst != self.name:
            print(f"[{self.name}] qubit de {src} redirecionado para {dst}")
            self.send_qubit(dst, photon)
            return

        # tenta alocar memória RAW
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                # isso é opcional pq é meio que de mentirinha, o quantummanager faz o trabalho de verdade
                info.memory.photon = photon
                print(f"[{self.name}] armazenou qubit de {src} em {info.memory.name}")
                return

        # se não encontrou memória, descarta o fóton
        print(f"[{self.name}] MEMÓRIA CHEIA. Qubit de {src} descartado.")
    
    def allocate_raw_memory(self):
        # TODO: verificar se esse é o mesmo caso do get, pra mim, parece diferente. não sei ainda por que
        memory = self.check_raw_memory()
        if memory is None:
            return 
        # Retorna a memória RAW disponível
        #self.resource_manager.update(None, memory, "OCCUPIED")
        memory.state = "OCCUPIED"
        return memory
    
    def check_raw_memory(self):
        # verifica se tem memória RAW disponível
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                return info.memory
        

In [5]:
from sequence.kernel.timeline import Timeline
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

# Assuma que estas classes estão definidas no seu código:
# - Hub
# - EntangleConnectionDispatcherProtocol
# - EntangleConectionHandshakeProtocol

def main():
    # Cria a timeline com duração de simulação
    tl = Timeline(1e12)
    tl.show_progress = True

    # Cria os dois nós
    node_a = Hub("A", tl)
    node_b = Hub("B", tl)
    node_a.set_seed(1)
    node_b.set_seed(2)

    # Conecta canais quânticos (bidirecional, mas só 1 sentido necessário se for simples)
    qc = QuantumChannel("qc_A_B", tl, attenuation=0, distance=1000)
    qc.set_ends(node_a, node_b.name)

    # Canal quântico de B para B (loopback interno)
    loopback_qc = QuantumChannel("qc_B_B", tl, attenuation=0, distance=0)
    loopback_qc.set_ends(node_b, node_b.name)

    loopback_cc = ClassicalChannel("cc_B_B", tl, distance=0, delay=1e9)
    loopback_cc.set_ends(node_b, node_b.name)

    
    # Conecta canais clássicos bidirecionais
    cc_ab = ClassicalChannel("cc_A_B", tl, distance=100, delay=1e9)
    cc_ba = ClassicalChannel("cc_B_A", tl, distance=100, delay=1e9)
    cc_ab.set_ends(node_a, node_b.name)
    cc_ba.set_ends(node_b, node_a.name)

    # Adiciona dispatcher ao nó B
    dispatcher_b = EntangleConnectionDispatcherProtocol(owner=node_b)
    node_b.protocols.append(dispatcher_b)

    # Cria a instância do protocolo de handshake no nó A (com B como destino)
    proto_ab = EntangleConectionHandshakeProtocol(owner=node_a, remote_node=node_b)
    node_a.protocols.append(proto_ab)

    # Inicializa a timeline
    tl.init()

    # Inicia o protocolo de entrelaçamento do lado A
    proto_ab.start()

    # Executa a simulação
    tl.run()

    print(f"Tempo final: {tl.now()}")

if __name__ == "__main__":
    main()


Recebido SYN de A com session_id 2173323925824
Tempo final: 2000000000
